# 🐯PyTorch版サンプルコード

公式のサンプルコードは2～3年前に作成されたもので、利用数が減ってきているTensorFlowが使われています。  
現在はPyTorchが主流であるため、このコードは、公式サンプルコードと同様の処理をPyTorch版に書き換えた物です。  
また、効率的に開発するため、PyTorchとセットで利用されることが多いPyTorch Lightningも使用しています。

このコードは、Google Colab（2024年8月21日時点）や、ローカルのDocker環境で動作確認済みです。

コード作成： PBL02「🐯とら」チーム CM_Kurozumi.

## 🔖環境の事前準備

Google Colab で動かしている場合は、以下の前準備処理を行います。  
Google Drive への接続確認画面が表示されたら、指示に従って接続してください。

- 必要なPythonパッケージをインストール（PyTorch Lightning と TorchMetrics）
- Google Drive への接続

In [ ]:
import sys

# Google Colab 上で実行しているかどうか
ON_COLAB = "google.colab" in sys.modules

# fmt: off
if ON_COLAB:
    print("Running on Google Colab.")
    # 必要なライブラリをインストール
    !pip install lightning==2.4.0 torchmetrics==1.4.1

    # Google Drive にマウント
    from google.colab import drive
    drive.mount("/content/drive")

else:
    print("Not running on Google Colab.")
# fmt: on

処理に必要なパッケージを読み込みます。  
主にPyTorch系のパッケージが中心になります。

In [ ]:
import os

import pandas as pd
import pytorch_lightning as pl
import torch
from PIL import Image
from pytorch_lightning.callbacks import Callback, EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger, WandbLogger
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchmetrics import Accuracy, F1Score, Precision, Recall
from torchvision import models, transforms
from torchvision.datasets import ImageFolder

## 🔖データフォルダの指定

利用環境に合わせて、学習などに利用する画像ファイルのデータフォルダ `DATA_DIR` と、  
学習時のログや結果を格納するログフォルダ `LOGS_DIR` を指定します。

Google Colab を使っている場合は、Google Drive に `DXQuest` というフォルダが作成され、  
その中に `train`, `test` などの画像データが格納されている想定にしています。

このパス情報は、利用している環境に合わせて適宜書き換えてください。

In [ ]:
# 利用するフォルダ名（Colabかどうかでパスを変更）
DATA_DIR = "/content/drive/MyDrive/DXQuest" if ON_COLAB else "/workspace/data"
LOGS_DIR = "/content/drive/MyDrive/logs" if ON_COLAB else "/workspace/logs"

print("Data folder:", DATA_DIR)
print("Logs folder:", LOGS_DIR)

## 🔖共通パラメータの設定

学習時の「バッチサイズ」や、クラス数の設定を行います。

また、内部で利用されている乱数のシード値を固定して、再実行時の再現性を高めています。  
乱数値のシードは `42` を指定していますが、この値に大きな意味は無いので、好きな数字にしてOKです。

In [ ]:
BATCH_SIZE = 32
NUM_CLASSES = 4

# 各種乱数の固定
pl.seed_everything(42)

## 🔖学習と検証データセットの定義

学習と検証に利用するデータに関する処理です。  
学習用と検証用に、それぞれ以下の処理を定義します。

- 実際に利用するデータの集まり（dataset）
- 各画像に対する加工処理（transform）
- データセットからデータを取り出す処理（data_loader）

💡学習データと検証データ

- 本来は、学習用と検証用のデータは「別のデータ」を利用します
- ただし、ここでは公式サンプルと同様に `train` のデータを検証でも利用するようにしています

In [ ]:
# データセットのディレクトリ
train_dir = os.path.join(DATA_DIR, "train")
val_dir = os.path.join(DATA_DIR, "train")  # 検証にもtrainデータを使う（本来は別のデータを使うべき）

# データ変換
data_transforms = transforms.Compose(
    [
        transforms.Resize((224, 224)),  # ResNet用に画像サイズを調整
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]
)

WORKERS = 2

# データセットとデータローダー（ラベルはフォルダ名から自動で設定される）
train_dataset = ImageFolder(train_dir, transform=data_transforms)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=WORKERS, pin_memory=True)

# Note:サンプルコードではvalidationデータもtrainデータと同じものを使っているが、本来は別のデータを使うべき！
val_dataset = ImageFolder(val_dir, transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=WORKERS, pin_memory=True)

`ImageFolder` を使ってデータセットを作成すると、フォルダ名で自動で正解ラベルが設定されます。  
実際に設定された「正解ラベルと、その値」は、datasetの `class_to_idx` で確認できます。

In [ ]:
# 自動で設定されたラベル番号とラベル名の確認
print(train_dataset.class_to_idx)

## 🔖モデルなどの定義

学習に利用するモデルの定義などを行います。  
ここでは、利用するモデルの種類や構造、損失関数、最適化手法などを定義します。  
また、学習の各ステップごとに行う処理も定義できます。

今回はResNet18を利用して、4クラス分類を行うモデルを作成します。

Accuracy, Precision, Recall, F1-Scoreなどの計算も、torchmetricsを使って簡単に計算できます。

In [ ]:
# モデル定義
class ResNetClassifier(pl.LightningModule):
    def __init__(self, num_classes=NUM_CLASSES):
        super(ResNetClassifier, self).__init__()
        self.model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.model.fc = nn.Sequential(
            nn.Linear(self.model.fc.in_features, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes),
        )

        self.criterion = nn.CrossEntropyLoss()
        self.accuracy = Accuracy(task="multiclass", num_classes=num_classes)
        self.precision = Precision(task="multiclass", num_classes=num_classes, average="weighted")
        self.recall = Recall(task="multiclass", num_classes=num_classes, average="weighted")
        self.f1 = F1Score(task="multiclass", num_classes=num_classes, average="weighted")

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log("train_loss", loss, on_epoch=True)
        self.log("train_acc", self.accuracy(outputs, labels), on_epoch=True)
        self.log("train_precision", self.precision(outputs, labels), on_epoch=True)
        self.log("train_recall", self.recall(outputs, labels), on_epoch=True)
        self.log("train_f1", self.f1(outputs, labels), on_epoch=True)
        return loss

    def validation_step(self, batch, batch_idx):
        images, labels = batch
        outputs = self(images)
        loss = self.criterion(outputs, labels)
        self.log("val_loss", loss, on_epoch=True)
        self.log("val_acc", self.accuracy(outputs, labels), on_epoch=True)
        self.log("val_precision", self.precision(outputs, labels), on_epoch=True)
        self.log("val_recall", self.recall(outputs, labels), on_epoch=True)
        self.log("val_f1", self.f1(outputs, labels), on_epoch=True)

    def configure_optimizers(self):
        # 最適化関数と学習率スケジューラーの設定（性能が向上しなくなったら学習率を減らす）
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-4)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode="min", factor=0.1, patience=3)
        return {"optimizer": optimizer, "lr_scheduler": scheduler, "monitor": "val_loss"}

    def predict_step(self, batch, batch_idx):
        images, _ = batch
        outputs = self(images)
        return torch.argmax(outputs, dim=1)


# 各エポックごとの結果を表示するコールバック
class PrintCallback(Callback):
    def on_train_epoch_end(self, trainer, pl_module):
        metrics = trainer.callback_metrics
        print(
            f" [{trainer.current_epoch:03}] "
            f"loss: {metrics['train_loss']:.4f}, "
            f"acc: {metrics['train_acc']:.4f}, "
            f"precision: {metrics['train_precision']:.4f}, "
            f"recall: {metrics['train_recall']:.4f}, "
            f"f1: {metrics['train_f1']:.4f}, "
            f"val_loss: {metrics['val_loss']:.4f}, "
            f"val_acc: {metrics['val_acc']:.4f}, "
            f"val_precision: {metrics['val_precision']:.4f}, "
            f"val_recall: {metrics['val_recall']:.4f}, "
            f"val_f1: {metrics['val_f1']:.4f}"
        )

## 🔖学習計画の準備

PyTorch Lightning を使って、学習の準備を行います。  
全体的な学習の管理を行うような「トレーナー」の設定になります。

`MAX_EPOCHS` の値を変更すれば、学習させる最大回数（エポック数）を変更できます。  
また、学習が進まなくなった場合は、そこで学習をストップさせるEarlyStoppingも設定しています。

💡EarlyStoppingに関する注意点
ここでは `val_loss`（検証用データに対する結果）が、3回連続で改善されない場合はストップさせる設定にしています。  
ただし、本サンプルコードでは、検証用データも「学習用データをそのまま使う」形にしているため、この状態では「val_lossに対する結果は常に良くなり続ける（そして過学習が起きる）」ため、EarlyStoppingが発動することは無いと思われます。  
EarlyStoppingは、学習用データと検証用データをきちんと分けた場合に効果を発揮します。


In [ ]:
MAX_EPOCHS = 10

# 学習時のログ記録（必要に応じてW&Bなどのログサービスを利用可能）
logger = None

# EarlyStoppingの設定
early_stopping = EarlyStopping(monitor="val_loss", patience=3, mode="min")

# Trainerの設定
trainer = pl.Trainer(
    default_root_dir=LOGS_DIR,
    max_epochs=MAX_EPOCHS,
    logger=logger,
    log_every_n_steps=5,
    callbacks=[early_stopping, PrintCallback()],
)

## 🔖学習

上記で定義したトレーナーが、自動で学習を進めていきます。  
学習に利用するデータ量やモデルの種類、環境などによって、学習時間は大きく異なります。

学習には数分～数時間程度の時間がかかります。

学習済みのモデルウェイトは `LOGS_DIR` で指定したフォルダ配下の、  
`lightning_logs/version_n/checkpoints` フォルダ配下に保存されます。

In [ ]:
# Tensor Coreが使える場合は利用する
# https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html
torch.set_float32_matmul_precision("high")

# モデルのトレーニング
model = ResNetClassifier()
trainer.fit(model, train_loader, val_loader)

## 🔖学習済みモデルの読み込み

学習したモデルウェイトを読み込みます。  
学習する度に、学習済みのウェイトファイル（*.ckpt）ファイルが作成されるため、ファイルをリストアップしています。

In [ ]:
# 学習済みのウェイトファイルをリストアップ
def enum_checkpoints(dir_path: str = LOGS_DIR) -> list:
    checkpoint_files = []
    for root, _, files in os.walk(dir_path):
        for file in files:
            if file.lower().endswith(".ckpt"):
                checkpoint_files.append(os.path.join(root, file))
    return sorted(checkpoint_files)


checkpoints = enum_checkpoints()
checkpoints

上記で取得したウェイトファイルの一覧から「最新のウェイト情報」を選んで読み込んでいます。  
最後のウェイトファイルが最も精度が高いとは限らないため、必要に応じて読み込むウェイトファイルは変更してください。

In [ ]:
# GPUが使える場合はGPUを使う（GPUがない場合はCPUを使う）
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# 学習済みのウェイト情報を読み込む
ckpt_file = checkpoints[-1]  # 最新のウェイトを読み込む（必要に応じて変更）
print("Checkpoint:", ckpt_file)
checkpoint = torch.load(ckpt_file)
model = ResNetClassifier()
model.load_state_dict(checkpoint["state_dict"])
model = model.to(device)

## 🔖テストデータの読み込み

test画像に対して推論していくため、テスト画像用の dataset, data_loader を定義します。  
画像を加工する処理は、trainなどで作成した transform をそのまま利用します。

testデータには正解ラベルが無い（regularなどの正解ラベルのフォルダが存在しない）ため、  
専用の `UnlabeledImageDataset` を定義します。（ImageFolderは正解ラベルのフォルダが無いと利用できないため）

In [ ]:
# ImageLoaderはラベル付きのデータセットを読み込むため、ラベルなしのデータセットを読み込むためのクラスを作成
class UnlabeledImageDataset(Dataset):
    def __init__(self, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_paths = [
            os.path.join(image_dir, img) for img in os.listdir(image_dir) if img.endswith((".png", ".jpg", ".jpeg"))
        ]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image


# /workspace/data/test から実際に推論する画像データを読み込み
test_dir = os.path.join(DATA_DIR, "test")
test_dataset = UnlabeledImageDataset(test_dir, transform=data_transforms)
test_loader = DataLoader(test_dataset, shuffle=False, batch_size=BATCH_SIZE, num_workers=WORKERS)

## 🔖テスト画像に対して推論

読み込んだテスト画像に対して、学習済みモデルで推論を行っていきます。  
推論結果は、各画像に対して 0～3 の学習時に使用した正解ラベルの番号（0=bridge など）で返ってきます。

全ての画像に対する推論結果を、リスト形式に成形して取得します。

In [ ]:
# テストデータに対して推論
all_preds = []
for i, images in enumerate(test_loader):
    images = images.to(device)
    with torch.no_grad():
        outputs = model(images)
        preds = torch.argmax(outputs, dim=1)
        all_preds.extend(preds.tolist())

print(all_preds)

## 🔖推論結果をDataFrameで整理

最終的には「ファイル名と、良品=0/不良品=1」の値をセットで出力する必要があるため、DataFrame型で表形式で整理します。

分かりやすいように、予測結果の数値（0～3）に対するラベルや、  
最終的な「regular=0（良品）、それ以外は1（不良品）」という形への変換も、ここで行います。


In [ ]:
# テストデータのパスと、推論結果をDataFrameにまとめる
df = pd.DataFrame({"path": test_dataset.image_paths, "pred": all_preds})

# ラベル、ファイル名、提出用の予測結果（良品=0/不良品=1）を追加
df["label"] = df["pred"].apply(lambda x: train_dataset.classes[x])  # ラベル（例: "regular"）
df["filename"] = df["path"].apply(lambda x: os.path.basename(x))  # ファイル名（例: "003.jpeg"）
df["y_hat"] = df["label"].apply(lambda x: 0 if x == "regular" else 1)  # 良品=0/不良品=1

# ファイル名の昇順にソートしてインデックスの振り直し
df = df.sort_values("filename").reset_index(drop=True)
df

## 🔖指定された形式でTSVファイルに保存

最終的に提出するTSVファイルを作成します。  
中身も「ファイル名と0/1」の値のセットをタブ区切りで出力する必要があるため、この形式で出力しています。

TSVファイルは `DATA_DIR` フォルダ配下に、 my_submission_{日時}.tsv というファイル名で出力されます。  
このファイルを、SIGNATE Cloud 経由で提出しましょう。

In [ ]:
# 保存するファイル名の設定（現在日時を付けておく）
now_str = pd.Timestamp.now(tz="Asia/Tokyo").strftime("%Y%m%d_%H%M%S")
tsv_filename = os.path.join(DATA_DIR, f"my_submission_{now_str}.tsv")

# filename, y_hat の項目だけをtsvファイルに保存（ヘッダ無し）
df[["filename", "y_hat"]].to_csv(tsv_filename, sep="\t", header=False, index=False)
print("Saved:", tsv_filename)